# 🚀 Analyse Rhétorique Collaborative par Agents IA - Exécuteur Principal

**Objectif:** Ce notebook orchestre et exécute une analyse rhétorique multi-agents sur un texte donné. Il sert de point d'entrée principal pour lancer le processus.

**Structure Modulaire:**
1.  `Argument_Analysis_UI_configuration.ipynb` : Gère l'interface utilisateur pour sélectionner/préparer le texte à analyser (incluant sources prédéfinies, URL, fichier, texte direct, et extraction) et charge/sauvegarde la configuration des sources.
2.  `Argument_Analysis_Agentic-0-init.ipynb`: Configuration initiale (dépendances, LLM, JVM), définition de l'état partagé (`RhetoricalAnalysisState`) et du gestionnaire d'état (`StateManagerPlugin`).
3.  `Argument_Analysis_Agentic-1-informal_agent.ipynb`: Définition de l'`InformalAnalysisAgent`.
4.  `Argument_Analysis_Agentic-2-pl_agent.ipynb`: Définition du `PropositionalLogicAgent`.
5.  `Argument_Analysis_Agentic-3-orchestration.ipynb`: Définition des stratégies d'orchestration et de la fonction principale `run_analysis_conversation`.

**Prérequis:**
* Un fichier `.env` à la racine contenant les clés API, configurations LLM, et la clé de chiffrement `TEXT_CONFIG_KEY`.
* Un environnement Java Development Kit (JDK >= 11) correctement installé et configuré (`JAVA_HOME`).
* Les dépendances Python installées (`ipywidgets`, `requests`, `jupyter-ui-poll`, `python-dotenv`, `semantic-kernel`, `pandas`, `jpype1`, `cryptography`).
* Les JARs Tweety placés dans le dossier `libs/`.
* Le fichier `extract_sources.json.gz.enc` (s'il existe déjà) contenant les définitions des sources.

## 1. Chargement de l'Environnement

Chargement des variables depuis le fichier `.env` (clés API, clé de chiffrement, etc.).

In [6]:
# Charger les variables d'environnement
from dotenv import load_dotenv, find_dotenv
loaded_env = load_dotenv(find_dotenv(), override=True)
print(f".env chargé: {loaded_env}") # Affiche True si le .env a été trouvé et chargé

.env chargé: True


## 2. Chargement de l'Interface Utilisateur

Exécution du notebook `UI_Configuration.ipynb` pour définir la fonction `configure_analysis_task()`. C'est ce notebook qui contient désormais toute la logique de l'interface graphique, du cache fichier et de la gestion de la configuration chiffrée.

In [7]:
# Exécuter le notebook UI pour définir la fonction configure_analysis_task
# Assurez-vous que le fichier UI_Configuration.ipynb est dans le même répertoire
# et qu'il contient bien la définition de la fonction configure_analysis_task()
print("Exécution de Argument_Analysis_UI_configuration.ipynb...")
%run ./Argument_Analysis_UI_configuration.ipynb
print("Exécution de Argument_Analysis_UI_configuration.ipynb terminée.")

# Vérification que la fonction est bien définie après l'exécution
if 'configure_analysis_task' not in locals():
    print("ERREUR CRITIQUE : La fonction configure_analysis_task n'a pas été définie par UI_Configuration.ipynb !")
    # Vous pourriez vouloir arrêter l'exécution ici
    # raise NameError("configure_analysis_task non définie")
else:
    print("Fonction configure_analysis_task trouvée.")

Exécution de Argument_Analysis_UI_configuration.ipynb...
Imports pour UI_Configuration chargés (incluant crypto KDF et base64).
Vérification de la phrase secrète 'TEXT_CONFIG_PASSPHRASE' dans .env...
✅ Phrase secrète trouvée. Dérivation de la clé...
✅ Clé de chiffrement dérivée et encodée.
Cache répertoire assuré : C:\dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\Argument_Analysis\text_cache
Configuration UI chargée. 4 sources initiales définies.
Fonctions utilitaires UI_Configuration définies.

--- Initialisation du Cache des Textes Complets ---
Vérification du cache pour 4 source(s)...

--- Fin Initialisation du Cache ---
✅ Cache initialisé/vérifié.
Fonction configure_analysis_task() définie dans UI_Configuration.ipynb.
Exécution de Argument_Analysis_UI_configuration.ipynb terminée.
Fonction configure_analysis_task trouvée.


## 3. Configuration de la Tâche et Récupération du Texte

Appel de la fonction `configure_analysis_task()` définie dans le notebook UI. Cela affichera l'interface utilisateur. Sélectionnez votre source, préparez le texte, puis cliquez sur **"Lancer l'Analyse"**. Le texte préparé sera retourné et stocké pour l'étape suivante. La cellule attendra la fin de votre interaction avec l'UI.

In [ ]:
# Appeler la fonction pour afficher l'UI et obtenir le texte
# Cette cellule bloquera jusqu'à ce que l'utilisateur valide dans l'UI

texte_pour_analyse = None # Initialiser
if 'configure_analysis_task' in locals():
    try:
        print("Lancement de l'interface de configuration...")
        texte_pour_analyse = configure_analysis_task() # Appel bloquant
    except Exception as e_ui:
        print(f"❌ Une erreur est survenue lors de l'exécution de l'interface utilisateur : {e_ui}")
        import traceback
        traceback.print_exc()
else:
    print("❌ ERREUR: Impossible d'appeler configure_analysis_task() car elle n'est pas définie.")


# Vérifier si on a bien reçu du texte après l'interaction UI
if not texte_pour_analyse:
    print("\n❌ Aucune tâche configurée ou texte préparé vide. L'analyse ne peut pas continuer.")
    # Optionnel : lever une exception pour arrêter proprement
    # raise ValueError("Configuration de la tâche échouée ou texte vide.")
else:
    print(f"\n✅ Texte prêt pour l'analyse (longueur: {len(texte_pour_analyse)}). Passage au chargement des définitions d'agents.")

Lancement de l'interface de configuration...
Fichier config 'extract_sources.json.gz.enc' non trouvé. Utilisation définitions en mémoire.
Initialisation interface...


Fichier config 'extract_sources.json.gz.enc' non trouvé. Utilisation définitions en mémoire.

⏳ En attente interaction...


## 4. Chargement des Définitions des Agents et de l'Orchestration

Maintenant que le texte est prêt (si l'étape précédente a réussi), nous chargeons les définitions des agents, des plugins, des stratégies et de la fonction d'orchestration `run_analysis_conversation` en exécutant les notebooks enfants dédiés.

**Rappel:** Le notebook `Argument_Analysis_Agentic-0-init.ipynb` **ne doit plus définir** la variable `raw_text_input` et le notebook `Argument_Analysis_Agentic-3-orchestration.ipynb` **doit définir** `run_analysis_conversation(texte_a_analyser)` acceptant un argument.

In [ ]:
# Exécuter les notebooks enfants pour charger les définitions
# Seulement si un texte a été préparé avec succès
if 'texte_pour_analyse' in locals() and texte_pour_analyse:
    print("\nChargement des définitions des agents et de l'orchestration...")
    try:
        %run ./Argument_Analysis_Agentic-0-init.ipynb
        %run ./Argument_Analysis_Agentic-1-informal_agent.ipynb
        %run ./Argument_Analysis_Agentic-2-pl_agent.ipynb
        %run ./Argument_Analysis_Agentic-3-orchestration.ipynb  # Définit run_analysis_conversation(texte_a_analyser)
        print("✅ Définitions chargées.")
        # Vérifier que la fonction d'orchestration est chargée
        if 'run_analysis_conversation' not in locals():
             print("❌ ERREUR CRITIQUE: La fonction run_analysis_conversation n'a pas été définie après l'exécution des notebooks agents!")
             # raise NameError("run_analysis_conversation non définie")
    except Exception as e_run:
        print(f"\n❌ Une erreur est survenue lors de l'exécution des notebooks enfants : {e_run}")
        import traceback
        traceback.print_exc()
        # Empêcher la suite si le chargement échoue
        texte_pour_analyse = None
else:
    print("\nSkipping agent definition loading because no text was prepared.")


Skipping agent definition loading because no text was prepared.


## 5. Exécution de l'Analyse Collaborative

Si toutes les étapes précédentes se sont bien déroulées et que nous avons un texte à analyser, cette cellule lance l'analyse collaborative.

*Note :* `nest_asyncio` est appliqué pour la compatibilité avec l'environnement asynchrone de Jupyter.

In [ ]:
import nest_asyncio
import asyncio

# Lancer seulement si on a obtenu un texte valide ET que les définitions sont chargées
if 'texte_pour_analyse' in locals() and texte_pour_analyse and 'run_analysis_conversation' in locals():
    print("\n🚀 Lancement de l'exécution asynchrone de l'analyse...")
    nest_asyncio.apply()
    try:
        # Passer le texte préparé
        await run_analysis_conversation(texte_pour_analyse)
        print("\n🏁 Analyse terminée.")
    except Exception as e_analysis:
        print(f"\n❌ Une erreur est survenue pendant l'exécution de l'analyse : {e_analysis}")
        import traceback
        traceback.print_exc()

elif 'texte_pour_analyse' not in locals() or not texte_pour_analyse:
    print("\n Analyse non lancée : aucun texte n'a été préparé ou une erreur est survenue avant.")
else: # Implique que run_analysis_conversation n'a pas été chargée
     print("\n Analyse non lancée : la fonction d'orchestration n'a pas pu être chargée.")


 Analyse non lancée : aucun texte n'a été préparé ou une erreur est survenue avant.


## 6. Résultats et Conclusion

Vérifiez les logs et l'état final JSON affichés par l'exécution précédente pour voir le résultat de l'analyse collaborative.

## 7. 🏁 Pistes d'Amélioration Futures

*(Repris de `3-orchestration-...ipynb`)*

**Prochaines étapes possibles :**
* **Activer & Finaliser PL:** Implémenter réellement les appels JPype/Tweety dans `PropositionalLogicPlugin._internal_execute_query` et tester de bout en bout l'exécution des requêtes logiques (parsing, query, interprétation).
* **Affiner Analyse Sophismes:** Améliorer les instructions de `InformalAnalysisAgent` pour une exploration plus fine de la taxonomie (gestion de la profondeur, choix des branches) ou l'attribution de sophismes spécifiques basée sur les détails récupérés (`get_fallacy_details`).
* **Externaliser Prompts & Config:** Déplacer les prompts et configurations (ex: noms agents, constantes) hors du code Python vers des fichiers dédiés (YAML, JSON, .env) pour une meilleure maintenabilité. Utiliser `kernel.import_plugin_from_directory`.
* **Gestion Erreurs Agents:** Renforcer la capacité des agents à gérer les erreurs retournées par les outils (`FUNC_ERROR:`) et à adapter leur plan (ex: demander une clarification, réessayer, passer à autre chose).
* **Nouveaux Agents/Capacités:** Implémenter des agents pour d'autres logiques (FOL, Modale), d'autres tâches (résumé, extraction d'entités) ou d'autres outils (recherche web, base de données).
* **État RDF/KG:** Explorer le passage à une structure d'état plus riche et sémantiquement structurée en utilisant RDF/KG (avec `rdflib` ou une base de graphe) pour représenter les arguments, relations, et métadonnées de manière plus formelle.
* **Interface Utilisateur:** Créer une interface (ex: avec Gradio, Streamlit) pour faciliter l'interaction et la visualisation de l'analyse.